In [82]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

True

In [83]:
import os
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [84]:
from pdf2image import convert_from_path
import os

# Function to convert PDF to images and save them
def pdf_to_images(pdf_path):
    # Convert PDF to images
    images = convert_from_path(pdf_path)
    output_folder = 'artifacts/'
    os.makedirs(output_folder, exist_ok= True)
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Save each image
    for i, image in enumerate(images):
        image_path = os.path.join(output_folder, f"page_{i + 1}.png")  # Save as PNG
        image.save(image_path, "PNG")  # You can change "PNG" to "JPEG" if needed
        # print(f"Saved: {image_path}")
        return image_path

In [85]:
import base64
import mimetypes
import shutil

# Function to convert image to base64
def image_to_base64(path):
    os.makedirs("artifacts/", exist_ok= True)
    _, extension = os.path.splitext(path)
    if extension == ".pdf":
        image_path = pdf_to_images(path)
    else: 
        image_path = path
        shutil.copy(image_path, "artifacts/")
    mime_type, _ = mimetypes.guess_type(image_path)
    with open(image_path, "rb") as image_file:
        base64_string = base64.b64encode(image_file.read()).decode("utf-8")
    base64_image = f"data:{mime_type};base64,{base64_string}"
    return base64_image

In [86]:
def generate_response(base64_path, input_prompt):
    llm = ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature= 0.8)
    extracted_inforamtion = HumanMessage(
    content=[
        {
            "type": "text", "text": input_prompt,},  
        {"type": "image_url", "image_url": image_to_base64(base64_path)},
    ]
)
    response = llm.invoke([extracted_inforamtion])
    # print(type(response))
    return response.content

In [87]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.8)

In [88]:
INPUT = """
You are an expert in understanding invoices.
You will receive input images as invoices &
You will have to extract the following information
Please extract important information from invoice, such as:
Name, Date, Description, Total Amount,etc
    """


In [89]:
path = "../Asus_bill.jpeg"

In [90]:
import json

In [97]:
response = generate_response(path, INPUT)

In [102]:
result= response.strip().replace('```json', '').replace('```', '').strip()

In [108]:
rrr= json.loads(result)


'Dreamscape Technologies'